<a href="https://colab.research.google.com/github/willystumblr/ict-2023-ems/blob/pv/%08ict_2023_model_predict_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torch import nn, optim
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split


In [ ]:
import glob
import tqdm
import os
from collections import defaultdict
import datetime
import time

In [ ]:
time.asctime()

'Thu Aug 10 11:24:48 2023'

In [ ]:
time.strftime('%Y-%m-%d %H:%M', time.localtime())

'2023-08-10 11:30'

In [ ]:
PATH = "/content/drive/MyDrive/ict-2023/train"
buildings = defaultdict(pd.DataFrame)
def parser(x):
    return datetime.datetime.strptime(x, '%Y %m %d %H')

for path in glob.glob(os.path.join(PATH, "*-revised.csv")):
    building = path.split("/")[-1][:-12]

    data = pd.read_csv(path, index_col=0, parse_dates=[['YEAR', 'MONTH', 'DAY', 'HOUR']], date_parser=parser)
    # dataset = read_csv('raw.csv',  parse_dates = [['year', 'month', 'day', 'hour']], index_col=0, date_parser=parse)
    t = pd.DataFrame(data.index)
    t_h = pd.DataFrame(data=list(t['YEAR_MONTH_DAY_HOUR'].dt.hour), index=data.index, columns=['Hour'])
    t_m = pd.DataFrame(data=list(t['YEAR_MONTH_DAY_HOUR'].dt.month), index=data.index, columns=['Month'])
    data = pd.concat([data, t_h, t_m], axis=1)
    buildings[building] = data

buildings[building].describe()

,시간당발전량(kWh),수평면(w/㎡),경사면(w/㎡),모듈온도(℃),기온(°C),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),일조(hr),일사(MJ/m2),전운량(10분위),시정(10m),Hour,Month
count,16128.000000,16128.000000,16128.000000,16128.000000,16128.000000,16128.000000,16128.000000,16128.000000,16128.000000,16128.000000,16128.000000,16128.000000,16128.000000,16128.000000,16128.00000
mean,15.871435,159.693576,173.885975,16.330742,14.570281,70.592603,14.239435,8.439745,1008.447576,0.260969,0.641556,5.469432,1615.363963,11.500000,6.27381
std,24.879874,252.472659,275.358798,15.310565,9.981969,22.868409,10.149309,11.744599,8.111703,0.402396,0.932334,3.847845,550.304083,6.922401,3.46885
min,0.000000,0.000000,0.000000,-19.300000,-13.400000,8.000000,0.600000,-28.800000,988.900000,0.000000,0.000000,0.000000,10.000000,0.000000,1.00000
25%,0.000000,0.000000,0.000000,3.700000,6.500000,54.000000,5.600000,-1.100000,1001.700000,0.000000,0.000000,1.000000,1300.000000,5.750000,3.00000
50%,0.000000,0.000000,0.000000,15.900000,15.100000,72.000000,10.300000,7.400000,1009.200000,0.000000,0.080000,7.000000,1956.000000,11.500000,6.00000
75%,23.900000,245.250000,260.000000,25.700000,23.300000,93.000000,23.100000,19.800000,1014.900000,0.600000,1.080000,9.000000,2000.000000,17.250000,9.00000
max,110.800000,1300.000000,1263.000000,63.300000,34.500000,99.000000,41.900000,29.800000,1027.500000,1.000000,3.880000,10.000000,12000.000000,23.000000,12.00000


In [ ]:
buildings.keys()

dict_keys(['산업협력관', '다산빌딩', '대학C동', '중앙창고', '시설관리동', '동물실험동', '중앙도서관', '삼성환경동', '학생회관', 'LG도서관', '학사과정', '신재생에너지동', '축구장', '기숙사B동'])

In [ ]:
buildings['기숙사B동'] = buildings['기숙사B동'].iloc[24:]

In [ ]:
buildings['기숙사B동'].shape, buildings['산업협력관'].shape

((16104, 15), (16104, 15))

In [ ]:
type(buildings['기숙사B동'].iloc[-24-24*7:-24])

pandas.core.frame.DataFrame

Store last sequence


In [ ]:
if not os.path.exists('/content/drive/MyDrive/ict-2023/eval_sequence'):
    os.makedirs('/content/drive/MyDrive/ict-2023/eval_sequence')

    for b in buildings.keys():
        buildings[b].iloc[-24-24*7:].to_csv('./eval_sequence/'+b+'-eval.csv', index=False)


# Data preprocessing

We first check the reformatted dataset combined with meteorological data.

We apply different encoding strategies to features according to their types; MinMaxScaler() for numerical; LabelEncoder() for categorical; and cyclic encoding for cyclic features.

In [ ]:
data.head()

,시간당발전량(kWh),수평면(w/㎡),경사면(w/㎡),모듈온도(℃),기온(°C),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),일조(hr),일사(MJ/m2),전운량(10분위),시정(10m),Hour_sin,Hour_cos,Month_sin,Month_cos
YEAR_MONTH_DAY_HOUR,,,,,,,,,,,,,,,,,
2020-10-24 00:00:00,0.0,0.0,0.0,9.2,11.0,54.0,7.1,2.0,1009.1,0.0,0.0,8.0,2000.0,0.000000,1.000000,-0.866025,0.5
2020-10-24 01:00:00,0.0,0.0,0.0,6.4,10.6,50.0,6.3,0.5,1009.1,0.0,0.0,0.0,2000.0,0.269797,0.962917,-0.866025,0.5
2020-10-24 02:00:00,0.0,0.0,0.0,6.3,10.0,53.0,6.5,0.8,1009.5,0.0,0.0,1.0,2000.0,0.519584,0.854419,-0.866025,0.5
2020-10-24 03:00:00,0.0,0.0,0.0,4.8,9.6,55.0,6.5,0.9,1009.4,0.0,0.0,5.0,2000.0,0.730836,0.682553,-0.866025,0.5
2020-10-24 04:00:00,0.0,0.0,0.0,4.4,9.7,54.0,6.5,0.8,1009.5,0.0,0.0,6.0,2000.0,0.887885,0.460065,-0.866025,0.5


Shape checking

# Build Model

We first define custom PyTorch dataset and build model for this task.

Since we are using LSTM for the prediction, we create sequences for training. We use the past 7 days' data to predict the next 24-hour solar power generation.

In [ ]:
class SolarDataset(Dataset):
    def __init__(self, dataframe, seq_len=7*24):
        self.seq_len = seq_len
        self.scaler = MinMaxScaler()
        self.y_scaler = MinMaxScaler()
        self.sequences, self.labels = self._preprocess(dataframe)




    def _preprocess(self, dataframe):

        # Separate numerical and categorical features
        numerical_features = ['시간당발전량(kWh)', '수평면(w/㎡)', '경사면(w/㎡)', '모듈온도(℃)', '기온(°C)', '습도(%)',
                           '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '일조(hr)',
                           '일사(MJ/m2)', '시정(10m)']
        categorical_features = ['전운량(10분위)']
        cyclic_features = ['Hour', 'Month']
        cyclic_preprocessed = ['Hour_sin', 'Hour_cos','Month_sin', 'Month_cos']

        # Create a Column Transformer to handle numerical and categorical columns
        preprocessor = ColumnTransformer(
        transformers=[
            ('num', self.scaler, numerical_features),
            ('cat', 'passthrough', categorical_features),
            ('cyc', 'passthrough', cyclic_preprocessed)
        ])


        # Perform cyclical encoding for cyclic features
        def cyclical_encode(df, feature, max_val):
            df[feature + '_sin'] = np.sin(2 * np.pi * df[feature] / max_val)
            df[feature + '_cos'] = np.cos(2 * np.pi * df[feature] / max_val)
            df = df.drop(feature, axis=1)
            return df

        # Apply cyclical encoding for each cyclic feature
        for feature in cyclic_features:
            max_val = dataframe[feature].max()
            dataframe = cyclical_encode(dataframe, feature, max_val)

        X_preprocessed = preprocessor.fit_transform(dataframe)
        y_preprocessed = self.y_scaler.fit_transform(dataframe['시간당발전량(kWh)'].to_numpy().reshape(-1,1))

        # Convert the preprocessed data into a PyTorch tensor
        X_tensor = torch.tensor(X_preprocessed, dtype=torch.float32)
        y_tensor = torch.tensor(y_preprocessed, dtype=torch.float32)

        # Create sequences and labels

        sequences = []
        labels = []

        if (len(X_tensor) - self.seq_len - 24)==0:
            sequences.append(X_tensor[:self.seq_len])
            labels.append(y_tensor[self.seq_len:self.seq_len+24])
        else:
            for i in range(len(X_tensor) - self.seq_len - 24):  # Subtract 24 to account for predicting the next 24 hours
                seq = X_tensor[i:i+self.seq_len]
                label = y_tensor[i+self.seq_len:i+self.seq_len+24]  # Next 24 hours
                sequences.append(seq)
                labels.append(label)

        return torch.stack(sequences), torch.stack(labels)


    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx].squeeze()

In [ ]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

## Arbitrary Hyperparameters (without tuning)

Set hyperparameters for the model.

In [ ]:
PATH = "/content/drive/MyDrive/ict-2023/output-tuned"

In [ ]:
# Set the device for training (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
input_size = 17
output_size = 24

configs = [ # combination of 3 hidden size, 4 num layers, and 2 epochs
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 100,
        "hidden_size": 64,
        "num_layers": 10,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 100,
        "hidden_size": 64,
        "num_layers": 9,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 100,
        "hidden_size": 64,
        "num_layers": 8,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 100,
        "hidden_size": 64,
        "num_layers": 7,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 100,
        "hidden_size": 32,
        "num_layers": 10,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 100,
        "hidden_size": 32,
        "num_layers": 9,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 100,
        "hidden_size": 32,
        "num_layers": 8,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 100,
        "hidden_size": 32,
        "num_layers": 7,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 100,
        "hidden_size": 128,
        "num_layers": 10,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 100,
        "hidden_size": 128,
        "num_layers": 9,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 100,
        "hidden_size": 128,
        "num_layers": 8,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 100,
        "hidden_size": 128,
        "num_layers": 7,
    },

    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 200,
        "hidden_size": 64,
        "num_layers": 10,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 200,
        "hidden_size": 64,
        "num_layers": 9,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 200,
        "hidden_size": 64,
        "num_layers": 8,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 200,
        "hidden_size": 64,
        "num_layers": 7,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 200,
        "hidden_size": 32,
        "num_layers": 10,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 200200,
        "hidden_size": 32,
        "num_layers": 9,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 200,
        "hidden_size": 32,
        "num_layers": 8,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 200,
        "hidden_size": 32,
        "num_layers": 7,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 200,
        "hidden_size": 128,
        "num_layers": 10,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 200,
        "hidden_size": 128,
        "num_layers": 9,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 200,
        "hidden_size": 128,
        "num_layers": 8,
    },
    {
    # Define the hyperparameters
        "batch_size": 256,
        "num_epochs": 200,
        "hidden_size": 128,
        "num_layers": 7,
    },
]

Set the criterion as MSELoss() to get RSME error.

In [ ]:
def load_data(dataframe, batch_size, num_workers=8):

    dataset = SolarDataset(dataframe)

    #8:1:1
    train_set, val_set = train_test_split(dataset, test_size=0.2, random_state=42)
    val_set, test_set = train_test_split(val_set, test_size=0.5, random_state=42)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8)
    val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=8)
    test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=8)

    return train_loader, val_loader, test_loader

In [ ]:
test = dict()
test = {"test":1, "eval":2, "train":3}
test.get('val')

In [ ]:
import pickle
RESULTS = "/content/drive/MyDrive/ict-2023/results/"

In [ ]:
def train(building, config, train_dataloader, val_dataloader, pklpath):
    checkpoint_path = os.path.join(PATH, building+'-checkpoint-{}.pt'.format(time.strftime('%Y-%m-%d %H:%M', time.localtime())))
    model = LSTMModel(input_size, config['hidden_size'], config['num_layers'], output_size).to(device)

    with open(pklpath, 'rb') as f:
        result = pickle.load(f)
    if result['Hyperparameters'] == config and os.path.exists(checkpoint_path):
        print(f"loading checkpoints from {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer = optim.Adam(model.parameters(), lr=checkpoint['lr'] if checkpoint.get('lr') else 1e-4)

    else:
        optimizer = optim.Adam(model.parameters(), lr=1e-4)

    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    criterion = nn.MSELoss()

    for epoch in range(0, config['num_epochs']):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = torch.sqrt(criterion(outputs, labels)) #RMSE Loss

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Update the learning rate
        scheduler.step()

        # Print the average loss for the epoch
        train_loss = running_loss / len(train_dataloader)


        model.eval()
        running_loss = 0.0

        with torch.no_grad():
            for inputs, labels in val_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Compute loss
                loss = torch.sqrt(criterion(outputs, labels))

                running_loss += loss.item()
        val_loss = running_loss / len(val_dataloader)

        if (epoch+1)%50==0:
            print(f"Epoch [{epoch+1}/{config['num_epochs']}] Train loss: {train_loss} Validation Loss: {val_loss}")


    # Save the model checkpoint
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'lr': optimizer.state_dict()['param_groups'][0]['lr'],
    }, checkpoint_path)
    print(f"Saved model to {checkpoint_path}")

    return model, optimizer

Now, we split the dataset into train and test set. We store each train-test sets pair to a seperate dictionary for convenience.

In [ ]:
def test(b, model, test_dataloader):
    model.eval()
    test_loss = 0.0
    criterion = nn.MSELoss()

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = torch.sqrt(criterion(outputs, labels))

            test_loss += loss.item()

    # Print the test loss
    test_loss /= len(test_dataloader)
    print(f"[{b}] Test Loss (RSME): {test_loss}")

    return test_loss

In [ ]:
import datetime
str(datetime.date.today())

'2023-08-08'

In [ ]:
def pv_predict(datapath: str, model):
    model.eval()

    df = pd.read_csv(datapath)
    dataset = SolarDataset(df)
    eval_loader = DataLoader(dataset=dataset, batch_size=1, shuffle=False)


    for seq, label in eval_loader:
        pred = model(seq.to(device))
        prediction_array = pred.detach().cpu().numpy()
        target_array = label.detach().cpu().numpy()
        prediction = dataset.y_scaler.inverse_transform(prediction_array)
        target = dataset.y_scaler.inverse_transform(target_array)
        rounded_data = np.maximum(prediction, 0.0).reshape(-1,1)

    return rounded_data.flatten(), target.flatten()

In [ ]:
buildings.keys()

dict_keys(['산업협력관', '다산빌딩', '대학C동', '중앙창고', '시설관리동', '동물실험동', '중앙도서관', '삼성환경동', '학생회관', 'LG도서관', '학사과정', '신재생에너지동', '축구장', '기숙사B동'])

In [ ]:
models = dict()
losses = dict()
optimizers = dict()
test_sets = dict()
predictions = dict()
targets = dict()

for config in configs:
    print(f"{config}")
    for b, eval in zip(sorted(buildings.keys()), sorted(glob.glob('/content/drive/MyDrive/ict-2023/eval_sequence/*.csv'))):
        print(f"+------- Training models for {b} -------+")
        train_dataloader, val_dataloader, test_loader = load_data(buildings[b], config['batch_size'])
        picklepath = sorted(glob.glob(RESULTS+"/"+b+"*.pkl"))[-1]
        model, optimizer = train(b, config, train_dataloader, val_dataloader, picklepath)

        print(f"+------- Testing models for {b}  -------+")
        loss = test(b, model, test_loader)
        models[b] = model
        losses[b] = loss
        optimizers[b] = optimizer

        print(f"+--------- Prediction for {b}  ---------+")
        predictions[b], targets[b] = pv_predict(eval, model)

    error = (pd.DataFrame(targets[b]) - pd.DataFrame(predictions[b])).sum().sum()
    print(f"Total Error for this config: {error}kWh")





{'batch_size': 256, 'num_epochs': 100, 'hidden_size': 64, 'num_layers': 10}
+------- Training models for LG도서관 -------+
Epoch [50/100] Train loss: 0.12279758944374616 Validation Loss: 0.1223334725946188
Epoch [100/100] Train loss: 0.12284319017265664 Validation Loss: 0.12233346328139305
Saved model to /content/drive/MyDrive/ict-2023/output-tuned/LG도서관-checkpoint-2023-08-10 13:10.pt
+------- Testing models for LG도서관  -------+
[LG도서관] Test Loss (RSME): 0.12121831811964512
+--------- Prediction for LG도서관  ---------+
+------- Training models for 기숙사B동 -------+


<ipython-input-82-80fcd63c8b2e>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature + '_sin'] = np.sin(2 * np.pi * df[feature] / max_val)
<ipython-input-82-80fcd63c8b2e>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature + '_cos'] = np.cos(2 * np.pi * df[feature] / max_val)


Epoch [50/100] Train loss: 0.13432473927736283 Validation Loss: 0.13571433297225408
Epoch [100/100] Train loss: 0.13433460384607315 Validation Loss: 0.13571423079286302
Saved model to /content/drive/MyDrive/ict-2023/output-tuned/기숙사B동-checkpoint-2023-08-10 13:17.pt
+------- Testing models for 기숙사B동  -------+
[기숙사B동] Test Loss (RSME): 0.13435601549489157
+--------- Prediction for 기숙사B동  ---------+
+------- Training models for 다산빌딩 -------+
Epoch [50/100] Train loss: 0.12936240630071671 Validation Loss: 0.1296529322862625
Epoch [100/100] Train loss: 0.1292887580443601 Validation Loss: 0.12965291738510132
Saved model to /content/drive/MyDrive/ict-2023/output-tuned/다산빌딩-checkpoint-2023-08-10 13:23.pt
+------- Testing models for 다산빌딩  -------+
[다산빌딩] Test Loss (RSME): 0.12814322672784328
+--------- Prediction for 다산빌딩  ---------+
+------- Training models for 대학C동 -------+
Epoch [50/100] Train loss: 0.1505474970966089 Validation

<ipython-input-82-80fcd63c8b2e>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature + '_sin'] = np.sin(2 * np.pi * df[feature] / max_val)
<ipython-input-82-80fcd63c8b2e>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature + '_cos'] = np.cos(2 * np.pi * df[feature] / max_val)


Epoch [50/100] Train loss: 0.13100406423211097 Validation Loss: 0.13234625543866838
Epoch [100/100] Train loss: 0.13097325652837755 Validation Loss: 0.13234622137887136
Saved model to /content/drive/MyDrive/ict-2023/output-tuned/기숙사B동-checkpoint-2023-08-10 14:53.pt
+------- Testing models for 기숙사B동  -------+
[기숙사B동] Test Loss (RSME): 0.1311967372894287
+--------- Prediction for 기숙사B동  ---------+
+------- Training models for 다산빌딩 -------+
Epoch [50/100] Train loss: 0.12769529907429805 Validation Loss: 0.12853130046278238
Epoch [100/100] Train loss: 0.12771415673807018 Validation Loss: 0.12853129301220179
Saved model to /content/drive/MyDrive/ict-2023/output-tuned/다산빌딩-checkpoint-2023-08-10 15:00.pt
+------- Testing models for 다산빌딩  -------+
[다산빌딩] Test Loss (RSME): 0.12593857385218143
+--------- Prediction for 다산빌딩  ---------+
+------- Training models for 대학C동 -------+
Epoch [50/100] Train loss: 0.135857418912356 Validation

...

<ipython-input-82-80fcd63c8b2e>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature + '_sin'] = np.sin(2 * np.pi * df[feature] / max_val)
<ipython-input-82-80fcd63c8b2e>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature + '_cos'] = np.cos(2 * np.pi * df[feature] / max_val)


Epoch [50/100] Train loss: 0.1963697463274002 Validation Loss: 0.1971567017691476
Epoch [100/100] Train loss: 0.19634904235601425 Validation Loss: 0.19715633136885508
Saved model to /content/drive/MyDrive/ict-2023/output-tuned/기숙사B동-checkpoint-2023-08-11 02:35.pt
+------- Testing models for 기숙사B동  -------+
[기숙사B동] Test Loss (RSME): 0.19420704245567322
+--------- Prediction for 기숙사B동  ---------+
+------- Training models for 다산빌딩 -------+
Epoch [50/100] Train loss: 0.1686681396648532 Validation Loss: 0.1676361095160246
Epoch [100/100] Train loss: 0.16873251268121062 Validation Loss: 0.16763592883944511
Saved model to /content/drive/MyDrive/ict-2023/output-tuned/다산빌딩-checkpoint-2023-08-11 02:45.pt
+------- Testing models for 다산빌딩  -------+
[다산빌딩] Test Loss (RSME): 0.16907824017107487
+--------- Prediction for 다산빌딩  ---------+
+------- Training models for 대학C동 -------+
Epoch [50/100] Train loss: 0.19851564481610157 Validation 

Exception ignored in: Exception ignored in: Exception ignored in: Exception ignored in: <function _after_fork at 0x7e07cd175990>Exception ignored in: 
<function _after_fork at 0x7e07cd175990>Traceback (most recent call last):
<function _after_fork at 0x7e07cd175990>
  File "/usr/lib/python3.10/threading.py", line 1635, in _after_fork
<function _after_fork at 0x7e07cd175990>
Traceback (most recent call last):
    Exception ignored in: thread._reset_internal_locks(False)  File "/usr/lib/python3.10/threading.py", line 1636, in _after_fork
<function _after_fork at 0x7e07cd175990><function _after_fork at 0x7e07cd175990>    

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1635, in _after_fork

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1635, in _after_fork
    Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1635, in _after_fork
thread._reset_internal_locks(False)    thread._stop()Ke

KeyboardInterrupt: ignored

: 
self._started._at_fork_reinit()
KeyboardInterrupt

:   File "/usr/lib/python3.10/threading.py", line 551, in _at_fork_reinit
    self._cond._at_fork_reinit()

  File "/usr/lib/python3.10/threading.py", line 260, in _at_fork_reinit
  File "/usr/lib/python3.10/threading.py", line 261, in _at_fork_reinit
    def _at_fork_reinit(self):    
self._lock._at_fork_reinit()KeyboardInterrupt: 

KeyboardInterrupt

Testing the models...

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/ict-2023/output/LG도서관-checkpoint-2023-08-08.pt')

In [ ]:
checkpoint['lr']

1.0000000000000024e-49

In [ ]:
# Save hyperparams
import pickle
RESULTS = "/content/drive/MyDrive/ict-2023/results/"

if not os.path.exists(RESULTS):
    os.makedirs(RESULTS)

for b in buildings.keys():
    scalers = y_scalers[b]

    metrics = {
        'RMSE': losses[b]
    }

    # Combine all results in one dictionary
    results = {
        'Hyperparameters': config,
        'Scalers': scalers,
        'Metrics': metrics
    }

    # Save results to a pickle file
    with open(f"{RESULTS}{b}-{str(datetime.date.today())}-hp.pkl", 'wb') as f:
        pickle.dump(results, f)


In [ ]:
os.listdir(RESULTS)

['중앙도서관-2023-08-05-hp.pkl',
 '대학C동-2023-08-05-hp.pkl',
 '다산빌딩-2023-08-05-hp.pkl',
 '학생회관-2023-08-05-hp.pkl',
 '동물실험동-2023-08-05-hp.pkl',
 '중앙창고-2023-08-05-hp.pkl',
 '산업협력관-2023-08-05-hp.pkl',
 '시설관리동-2023-08-05-hp.pkl',
 '삼성환경동-2023-08-05-hp.pkl',
 'LG도서관-2023-08-05-hp.pkl',
 '신재생에너지동-2023-08-05-hp.pkl',
 '기숙사B동-2023-08-05-hp.pkl',
 '축구장-2023-08-05-hp.pkl',
 '학사과정-2023-08-05-hp.pkl',
 '산업협력관-2023-08-08-hp.pkl',
 '다산빌딩-2023-08-08-hp.pkl',
 '대학C동-2023-08-08-hp.pkl',
 '중앙창고-2023-08-08-hp.pkl',
 '시설관리동-2023-08-08-hp.pkl',
 '동물실험동-2023-08-08-hp.pkl',
 '중앙도서관-2023-08-08-hp.pkl',
 '삼성환경동-2023-08-08-hp.pkl',
 '학생회관-2023-08-08-hp.pkl',
 'LG도서관-2023-08-08-hp.pkl',
 '학사과정-2023-08-08-hp.pkl',
 '신재생에너지동-2023-08-08-hp.pkl',
 '축구장-2023-08-08-hp.pkl',
 '기숙사B동-2023-08-08-hp.pkl']

In [ ]:
# Save results to a pickle file
with open(f"{RESULTS}{b}-{str(datetime.date.today())}-hp.pkl", 'rb') as f:
    loaded = pickle.load(f)
loaded

{'Hyperparameters': {'batch_size': 256,
  'num_epochs': 100,
  'hidden_size': 256,
  'num_layers': 10},
 'Scalers': MinMaxScaler(),
 'Metrics': {'RMSE': 0.23477265238761902}}

## Hyperparameter-tuning version

In [ ]:
!pip install 'ray[tune]'

In [ ]:
!pip install wandb

In [ ]:
from ray import tune
from ray.air import Checkpoint, session
from ray.tune.schedulers import ASHAScheduler

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import ray
ray.init(ignore_reinit_error=True, num_cpus=12, num_gpus=1)

2023-08-07 09:11:49,128	INFO worker.py:1621 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.6.2


In [ ]:
from ray.air.integrations.wandb import WandbLoggerCallback
from ray.tune.logger import DEFAULT_LOGGERS

In [ ]:
os.environ["FUNCTION_SIZE_ERROR_THRESHOLD"] = "400"

In [ ]:
sequences, labels = seq_dict[b], lab_dict[b]

In [ ]:
# Ray Tune configuration
config = {
    "num_epochs": 100,
    "lr": tune.loguniform(1e-4, 1e-1),
    "num_layers": tune.choice([5,6,7,8]),
    "batch_size": tune.choice([32, 64, 128]),
    "hidden_size": tune.choice([32, 64, 128]),
    "wandb": {
        "log_config": True,
        "api_key": "99911b0ddfa05a10619f53353db2a824461aca36",
        "project": "ict-2023"
    }
}

In [ ]:
# Set the device for training (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the hyperparameters
input_size = 17  # Number of features
output_size = 24

In [ ]:
PATH = "/content/drive/MyDrive/ict-2023/output-tuned"
if not os.path.exists(PATH):
    os.makedirs(PATH)

In [ ]:
def train_model(config):

    train_dataloader, val_dataloader, test_dataloader = load_data(sequences, labels, config["batch_size"])

    model = LSTMModel(input_size, config["hidden_size"], config["num_layers"], output_size).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

    checkpoint = session.get_checkpoint()

    if checkpoint:
        checkpoint_state = checkpoint.to_dict()
        start_epoch = checkpoint_state["epoch"]
        model.load_state_dict(checkpoint_state["model_state_dict"])
        optimizer.load_state_dict(checkpoint_state["optimizer_state_dict"])
    else:
        start_epoch = 0



    for epoch in range(config['num_epochs']):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute loss
            loss = torch.sqrt(criterion(outputs, labels)) #RMSE Loss

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()


            # Print the average loss for the epoch
        train_loss = running_loss / len(train_dataloader)


        model.eval()
        running_loss = 0.0

        with torch.no_grad():
            for inputs, labels in val_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                    # Forward pass
                outputs = model(inputs)

                    # Compute loss
                loss = torch.sqrt(criterion(outputs, labels))

                running_loss += loss.item()
        val_loss = running_loss / len(val_dataloader)

            # Here we save result for this step of training.

        checkpoint_data = {
            "epoch": epoch,
            "net_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        }
        checkpoint = Checkpoint.from_dict(checkpoint_data)

        session.report(
            {"loss": val_loss},
            checkpoint=checkpoint,
        )

    print("Finished Training")



In [ ]:
from ray import air

scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=100,
        grace_period=1,
        reduction_factor=2,
    )


analysis = tune.run(
    train_model,
    config=config,
    resources_per_trial={"cpu": 1, "gpu": 0.1},
    num_samples= 20,
    scheduler = scheduler,
    callbacks=[WandbLoggerCallback(
        project="ict-2023",
        api_key='99911b0ddfa05a10619f53353db2a824461aca36',
        log_config=True
    )]
)
best_trial = analysis.get_best_trial("loss", "min", "last")
print("Best config: ", analysis.get_best_config(metric="loss", mode="min"))


# The Next 24-hour Prediction

In [ ]:
CHECKPOINT_PATH = "/content/drive/MyDrive/ict-2023/output"

In [ ]:
glob.glob(CHECKPOINT_PATH+"/*-checkpoint-*.pt")[0].split("/")[-1].split(".")[0][:-11]

'중앙도서관'

In [ ]:
# Load the saved model checkpoint
modelsdict = dict()
for path in glob.glob(CHECKPOINT_PATH+"/*-checkpoint-*.pt"):
    building = path.split("/")[-1].split(".")[0][:-11]
    model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
    model.load_state_dict(torch.load(path))
    modelsdict[building] = model

In [ ]:
modelsdict.keys()

dict_keys(['중앙도서관', '시설관리동', '산업협력관', '동물실험동', '신재생에너지동', '기숙사B동', '대학C동', 'LG도서관', '중앙창고', '학생회관', '학사과정', '삼성환경동', '축구장', '다산빌딩'])

The prediction based on the last 7 days' data

In [ ]:
pred_PATH = "/content/drive/MyDrive/ict-2023/pred"
data_all = dict() # dataframe for display
for b in modelsdict.keys():
    valid_sequences = []
    valid_sequences.append(X_dict[b][-24*7:]) # predict based on the last 7 days' data
    valid_sequences = torch.stack(valid_sequences)
    valid_sequences.shape

    model = modelsdict[b]
    scaler = scalers[b]
    model.eval()
    prediction = model(valid_sequences.to(device))

    # Convert the predictions tensor to a numpy array
    predictions_array = prediction.detach().cpu().numpy()
    prediction_array = scaler.inverse_transform(predictions_array)

    #convert negative value to positive
    rounded_data = np.maximum(prediction_array, 0.0).reshape(-1,1)

    data_all[b] = rounded_data.flatten()

    # Create a DataFrame from the predictions array
    predictions_df = pd.DataFrame(rounded_data, columns=['Prediction'])
    predictions_df.to_csv(os.path.join(pred_PATH, b+"-revised.csv"), index=False)

In [ ]:
pd.DataFrame(data_all).to_csv(os.path.join(pred_PATH,'summed.csv'), index=False)
pd.DataFrame(data_all)

In [ ]:
all = pd.DataFrame(data_all)
all.to_csv("/content/drive/MyDrive/ict-2023/pred/merged.csv", index=False)